In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Configure pandas display settings
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# Set random seed for reproducibility
np.random.seed(42)
n_samples = 1000

# Create synthetic dataset with intentional errors and edge cases
data = {
    # Customer ID with blank spaces for every 10th record
    'Customer_ID': [f'CUS_{i:03d}' if i % 10 != 0 else ' ' for i in range(n_samples)],

    # Names with NaN for every 5th record
    'Name': [f'Customer {i}  ' if i % 5 != 0 else np.nan for i in range(n_samples)],

    # Emails with invalid entries and NaN values
    'Email': [
        f'customer{i}@email.com' if i % 7 != 0
        else ('invalid_email' if i % 14 == 0 else np.nan)
        for i in range(n_samples)
    ],

    # Ages with invalid ranges and NaN values
    'Age': [
        np.random.randint(18, 90) if i % 8 != 0
        else (np.random.randint(-10, 150) if i % 16 == 0 else np.nan)
        for i in range(n_samples)
    ],

    # Purchase amounts with invalid ranges and NaN values
    'Purchase_Amount': [
        np.random.uniform(10, 1000) if i % 6 != 0
        else (np.random.uniform(-100, 5000) if i % 12 == 0 else np.nan)
        for i in range(n_samples)
    ],

    # Purchase dates with invalid formats
    'Purchase_Date': [
        (datetime.now() - pd.Timedelta(days=np.random.randint(0, 365))).strftime('%Y-%m-%d %H:%M:%S')
        if i % 9 != 0 else 'invalid_date'
        for i in range(n_samples)
    ],

    # Product categories with blank spaces
    'Product_Category': [
        np.random.choice(['Electronics', 'Clothing', 'Books', 'Home & Garden'])
        if i % 11 != 0 else ' '
        for i in range(n_samples)
    ]
}

# Create initial DataFrame
df = pd.DataFrame(data)

# Add duplicate records for testing
duplicate_indices = np.random.choice(n_samples, size=50, replace=False)
df = pd.concat([df, df.iloc[duplicate_indices]], ignore_index=True)

# Step 1: Create a copy and remove duplicates
copy = df.copy().drop_duplicates()
df_new = copy

# Step 2: Initial data type conversions and cleaning
# Strip whitespace from Product Category
df_new['Product_Category'] = df_new['Product_Category'].str.strip()
# Convert Age to numeric, invalid values become NaN
df_new['Age'] = pd.to_numeric(df_new['Age'], errors='coerce')
# Convert Purchase Date to datetime format
df_new['Purchase_Date'] = pd.to_datetime(df_new['Purchase_Date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
# Convert Product Category to categorical type
df_new['Product_Category'] = df_new['Product_Category'].astype('category', errors='ignore')

# Step 3: Convert discrepancies to NULL values and clean strings
# Replace blank spaces and invalid values with NaN
df_new.loc[df_new['Customer_ID'] == ' ', 'Customer_ID'] = np.nan
df_new.loc[df_new['Purchase_Date'] == 'invalid_date', 'Purchase_Date'] = np.nan
df_new.loc[df_new['Product_Category'] == ' ', 'Product_Category'] = np.nan
df_new.loc[df_new['Email'] == 'invalid_email', 'Email'] = np.nan

# Strip whitespace from Name column and handle NaN values
df_new['Name'] = df_new['Name'].str.strip()
df_new['Name'] = df_new['Name'].fillna(np.nan)

# Step 4: Function to correct Customer IDs based on email addresses
def correct_id(email):
    """Extract customer number from email and generate correct Customer ID format"""
    try:
        number = re.findall(r'\d+', email)[0]
        number = int(number)
        return f'CUS_{number:03d}'
    except:
        return None

# Apply Customer ID correction
df_new.loc[df_new['Customer_ID'].isnull(), 'Customer_ID'] = (
    df_new.loc[df_new['Customer_ID'].isnull(), 'Email'].apply(correct_id)
)

# Step 5: Data validation functions
def validate_age(dat):
    """Validate age is between 18 and 90"""
    age_mask = (dat['Age'] >= 18) & (dat['Age'] <= 90)
    dat.loc[~age_mask, 'Age'] = np.nan
    return dat

def validate_purchase_amount(dat):
    """Validate purchase amount is between 10 and 1000"""
    amount_mask = (dat['Purchase_Amount'] >= 10) & (dat['Purchase_Amount'] <= 1000)
    dat.loc[~amount_mask, 'Purchase_Amount'] = np.nan
    return dat

def impose_validation(dataFrame):
    """Apply all validation rules"""
    dataFrame = validate_age(dataFrame)
    dataFrame = validate_purchase_amount(dataFrame)
    return dataFrame

# Apply validations
df_new = impose_validation(copy)

# Step 6: Fill missing emails based on Customer ID
def fetching_num(id):
    """Generate email address from Customer ID"""
    try:
        number = re.findall(r'\d+', id)[0]
        number = int(number)
        return f'customer{number}@email.com'
    except:
        return None

df_new.loc[df_new['Email'].isnull(), 'Email'] = (
    df_new.loc[df_new['Email'].isnull(), 'Customer_ID'].apply(fetching_num)
)

# Step 7: Fill missing names based on email
def name_num(email):
    """Generate customer name from email"""
    try:
        number = re.findall(r'\d+', email)[0]
        return f'Customer {number}'
    except:
        return None

df_new.loc[df_new['Name'].isnull(), 'Name'] = (
    df_new.loc[df_new['Name'].isnull(), 'Email'].apply(name_num)
)

# Step 8: Fill missing purchase amounts with category means
mean_purchase_amount = df_new.groupby('Product_Category')['Purchase_Amount'].mean()
mean_byAll = df_new['Purchase_Amount'].mean()

def mean_amt(category):
    """Return mean purchase amount for given category or overall mean"""
    try:
        if category in mean_purchase_amount.index:
            return mean_purchase_amount[category]
        return mean_byAll
    except:
        return None

df_new.loc[df_new['Purchase_Amount'].isnull(), 'Purchase_Amount'] = (
    df_new.loc[df_new['Purchase_Amount'].isnull(), 'Product_Category']
    .apply(mean_amt)
    .astype(float)
)

# Step 9: Handle missing product categories
df_new['Product_Category'] = df_new['Product_Category'].cat.add_categories(['Uncategorized'])
df_new['Product_Category'].fillna('Uncategorized', inplace=True)

# Step 10: Fill missing ages with mean age
mean_age = np.abs(df_new['Age'].mean())
df_new['Age'].fillna(mean_age, inplace=True)
df_new['Age'] = df_new['Age'].astype(int)

# Step 11: Remove rows with missing purchase dates
df_new = df_new.dropna(subset=['Purchase_Date'])

# Step 12: Remove rows where all identification fields are null
null_indices = df_new[df_new[['Customer_ID', 'Name', 'Email']].isnull().sum(axis=1) == 3].index
df_new = df_new.drop(index=null_indices)

# Step 13: Generate data cleaning report
def data_report(original_df, cleaned_df):
    """Generate report comparing original and cleaned datasets"""
    report = {
        'Original rows': len(original_df),
        'Cleaned rows': len(cleaned_df),
        'Rows removed': len(original_df) - len(cleaned_df),
        'null_values_filled': original_df.isnull().sum() - cleaned_df.isnull().sum()
    }
    return pd.DataFrame(report)

# Generate final report
report = data_report(df, df_new)
print(report)

# Optional: Export cleaned data
# df_new.to_excel('cleaned_data.xlsx', index=False)